In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
import random
import os

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device",device)

Using Device cuda


 ## Hyperparameters

In [4]:
BATCH_SIZE=16
BLOCK_SIZE=64
MAX_ITERS=10000
EVAL_INTERVAL=500
LEARNING_RATE=3e-4
EMBEDDING_DIM=256
NUM_HEADS=8
NUM_LAYERS=6
DROPOUT=0.2

## Dataset - Tinyshakespeare

In [5]:
with open("Tinyshakespeare.txt","r",encoding="utf-8") as f:
    text=f.read()

print(text[:1000])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



## Data Preprocessing

In [6]:
chars=sorted(list(set(text)))
vocab_size = len(chars)
stoi={ch: i for i,ch in enumerate(chars)}
itos={i: ch for i,ch in enumerate(chars)}
encode=lambda s: [stoi[c] for c in s]
decode=lambda l: ''.join([itos[i] for i in l])

## Train-Validation split

In [7]:
data =  torch.tensor(encode(text), dtype=torch.long)
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

## Data Loading

In [8]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data)-BLOCK_SIZE,(BATCH_SIZE,))
    x=torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y=torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x,y

## Transformer Components

### Self-Head Attention

In [9]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key=nn.Linear(EMBEDDING_DIM,head_size,bias=False)
        self.query=nn.Linear(EMBEDDING_DIM,head_size,bias=False)
        self.value=nn.Linear(EMBEDDING_DIM,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(BLOCK_SIZE,BLOCK_SIZE)))
        self.dropout=nn.Dropout(DROPOUT)

    def forward(self,x):
        B,T,C=x.shape
        k=self.key(x)
        q=self.query(x)
        wei = q @ k.transpose(-2,-1) * (k.size(-1) ** -0.5)
        wei=wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
        wei=torch.softmax(wei,dim=-1)
        wei=self.dropout(wei)
        v=self.value(x)
        out=wei@v

        return out


### Multi-Head Attention

In [10]:
class MultiHeadAttention(nn.Module):
    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, EMBEDDING_DIM)
        self.dropout=nn.Dropout(DROPOUT)

    def forward(self,x):
        out=torch.cat([h(x) for h in self.heads],dim=-1)
        out = self.proj(out)
        out=self.dropout(out)
        return out

### Feed Forward

In [11]:
class feedforward(nn.Module):
    def __init__(self,n_embd):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(DROPOUT)
        )

    def forward(self,x):
        return self.net(x)

### Transformer block

In [12]:
class Block(nn.Module):
    def __init__(self,n_embd,n_head):
        super().__init__()
        head_size=n_embd//n_head
        self.sa=MultiHeadAttention(n_head,head_size)
        self.ffwd=feedforward(n_embd)
        self.l1=nn.LayerNorm(n_embd)
        self.l2=nn.LayerNorm(n_embd)

    def forward(self,x):
        x=x+self.sa(self.l1(x))
        x=x+self.ffwd(self.l2(x))

        return x

## Mini-GPT Model

In [13]:
class MiniGPTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,EMBEDDING_DIM)
        self.position_embedding_table=nn.Embedding(BLOCK_SIZE,EMBEDDING_DIM)
        self.block=nn.Sequential(*[Block(EMBEDDING_DIM,n_head=NUM_HEADS) for i in range(NUM_LAYERS)])
        self.ln_f=nn.LayerNorm(EMBEDDING_DIM)
        self.lm_head=nn.Linear(EMBEDDING_DIM,vocab_size)

    def forward(self,idx,targets=None):
        B,T=idx.shape
        tok_emb=self.token_embedding_table(idx)
        pos_emb=self.position_embedding_table(torch.arange(T,device=device))
        x=tok_emb+pos_emb
        x=self.block(x)
        x=self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss=None
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=nn.functional.cross_entropy(logits,targets)

        return logits,loss

    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond=idx[:,-BLOCK_SIZE:]
            logits,loss=self(idx_cond)
            logits = logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

model = MiniGPTModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):
    if iter % EVAL_INTERVAL == 0:
        losses = {}
        model.eval()
        for split in ['train', 'val']:
            losses[split] = 0
            for _ in range(10):
                x, y = get_batch(split)
                logits, loss = model(x, y)
                losses[split] += loss.item()
            losses[split] /= 10
        print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        model.train()


    xb, yb = get_batch('train')


    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("\nGenerated text:")
print(decode(model.generate(context, max_new_tokens=200)[0].tolist()))

Step 0: train loss 4.3765, val loss 4.3751
Step 500: train loss 2.1480, val loss 2.1981
Step 1000: train loss 1.9123, val loss 2.0041
Step 1500: train loss 1.7861, val loss 1.9088
Step 2000: train loss 1.6831, val loss 1.8855
Step 2500: train loss 1.6178, val loss 1.8019
Step 3000: train loss 1.5647, val loss 1.7713
Step 3500: train loss 1.5316, val loss 1.7513
Step 4000: train loss 1.5199, val loss 1.6936
Step 4500: train loss 1.5045, val loss 1.6734
Step 5000: train loss 1.5116, val loss 1.6806
Step 5500: train loss 1.4701, val loss 1.7159
Step 6000: train loss 1.4383, val loss 1.6654
Step 6500: train loss 1.4436, val loss 1.6168
Step 7000: train loss 1.3866, val loss 1.6225
Step 7500: train loss 1.4198, val loss 1.6296
Step 8000: train loss 1.3924, val loss 1.6317
Step 8500: train loss 1.3744, val loss 1.6205
Step 9000: train loss 1.3589, val loss 1.5628
Step 9500: train loss 1.3786, val loss 1.6070

Generated text:

And his friard masters doth made and needs:
She have it beety time

In [14]:

prompt ="Hello"

context = torch.tensor([encode(prompt)], dtype=torch.long, device=device)  # shape [1, T]

generated_idx = model.generate(context, max_new_tokens=1000)
generated_text = decode(generated_idx[0].tolist())
print(generated_text)


Hellow let entry: some rage on show men? I shall tell
And me at oven his true, knows up plaim quence; tell
Of for the worward, and clockning his state,
and endrage the crodplentamned will well pure till I have.

CAPULET:
Then signora! what long you self, you see! If the ple,
Dost you father's the necess against the poert:
Wherefore are son? am kite you would sey?

QUEEN ELIZABETER:
And cousins not his son be, so, Feel me green.

HENRY BOLINGBROKE:
Did tell I was before out witness;
Which wigh it that abody too so?
Therefore I sap thy insce'stroofs me.

BARICHARD:
Well, whereout, what word, be he putinary on the sion
That despiral tring.

All Fair:
What is thither to as the peacefulles of the words:
Ay, gainst all to-glages that you good hold by the
shepherds my love griefs naturess;
You are my excutor hath love late men;
Which reason shows to-boll'd that had four this
Sen ded my proving as you kind again! The voices will
Amongher furready;
Indeedful delem.

PRINCE:
Beseech you have us 

In [15]:
torch.save(model.state_dict(), "model.pth")
print("Model saved as model.pth")

Model saved as model.pth
